# 1. 각 패키지 설치 및 불러오기

In [181]:
from PIL import Image
import glob, os
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow import keras

# 2. resize하기

In [188]:
def resize_images(img_path):
    images=glob.glob(img_path + "/*.jpg")  
    
    print(len(images), " images to be resized.")

  # 이미지를 불러와 파일마다 모두 122사이즈로 바꾸어 저장
    target_size=(122,122)
    for img in images:
        old_img=Image.open(img)
        new_img=old_img.resize(target_size,Image.ANTIALIAS)
        new_img.save(img, "JPEG")
    
directories = ['rock', 'scissor', 'paper']  # 3개 폴더 모두 적용

# 가위 바위 보 3 폴더에 대해서 resize 진행
for dir in directories:   # rock, scissor, paper 3개 폴더에 대해서
    images = glob.glob(train_images_path + "/" + dir + "/*.jpg")  # 폴더 안의 모든 사진을
    for img in images:
        resize_image(img)   # resize
print("rock scissor paper resize finished")

rock scissor paper resize finished


In [189]:
def load_data(img_path, number_of_data=1500):  # 가위바위보 이미지 개수 총합에 주의하세요.
    #라벨링 -> 가위 : 0, 바위 : 1, 보 : 2
    img_size=122
    color=3
    
    #이미지 데이터와 라벨(가위 : 0, 바위 : 1, 보 : 2) 데이터를 담을 행렬(matrix) 영역을 생성합니다.
    imgs=np.zeros(number_of_data*img_size*img_size*color,dtype=np.int32).reshape(number_of_data,img_size,img_size,color)
    labels=np.zeros(number_of_data,dtype=np.int32)

    idx=0
    for file in glob.iglob(img_path+'/scissor/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=0   # 가위 : 0
        idx=idx+1

    for file in glob.iglob(img_path+'/rock/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=1   # 바위 : 1
        idx=idx+1  
    
    for file in glob.iglob(img_path+'/paper/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=2   # 보 : 2
        idx=idx+1
        
    print("학습데이터(x_train)의 이미지 개수는", idx,"입니다.")
    return imgs, labels

image_dir_path = os.getenv("HOME") + "/aiffel/rock_scissor_paper/train"
(x_train, y_train)=load_data(image_dir_path)
x_train_norm = x_train/255.0   # 입력은 0~1 사이의 값으로 정규화

print("x_train shape: {}".format(x_train.shape))
print("y_train shape: {}".format(y_train.shape))

학습데이터(x_train)의 이미지 개수는 1500 입니다.
x_train shape: (1500, 122, 122, 3)
y_train shape: (1500,)


# 3. 네트워크 설계하기

In [190]:
model = keras.models.Sequential()
model.add(keras.layers.Conv2D(64, (3, 3), activation = 'relu', input_shape=(122, 122, 3)))
model.add(keras.layers.MaxPooling2D(2, 2))
model.add(keras.layers.Conv2D(128, (3, 3), activation = 'relu'))
model.add(keras.layers.MaxPooling2D(2, 2))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(256, activation = 'relu')) 
model.add(keras.layers.Dense(3, activation = 'softmax'))

model.summary()

Model: "sequential_18"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_34 (Conv2D)           (None, 120, 120, 64)      1792      
_________________________________________________________________
max_pooling2d_32 (MaxPooling (None, 60, 60, 64)        0         
_________________________________________________________________
conv2d_35 (Conv2D)           (None, 58, 58, 128)       73856     
_________________________________________________________________
max_pooling2d_33 (MaxPooling (None, 29, 29, 128)       0         
_________________________________________________________________
flatten_16 (Flatten)         (None, 107648)            0         
_________________________________________________________________
dense_32 (Dense)             (None, 256)               27558144  
_________________________________________________________________
dense_33 (Dense)             (None, 3)               

# 4. 네트워크 학습

In [191]:
model.compile(optimizer='adam', 
              loss='sparse_categorical_crossentropy', 
              metrics=['accuracy'])
model.fit(x_train_norm, y_train, epochs=10)

Epoch 1/10
47/47 [==============================] - 2s 27ms/step - loss: 0.3310 - accuracy: 0.9440
Epoch 2/10
47/47 [==============================] - 1s 27ms/step - loss: 4.7684e-10 - accuracy: 1.0000
Epoch 3/10
47/47 [==============================] - 1s 27ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 4/10
47/47 [==============================] - 1s 27ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 5/10
47/47 [==============================] - 1s 27ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 6/10
47/47 [==============================] - 1s 27ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 7/10
47/47 [==============================] - 1s 27ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 8/10
47/47 [==============================] - 1s 27ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 9/10
47/47 [==============================] - 1s 27ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 10/10
47/47 [==============================] - 1s 27ms/step - loss: 0.0

# 4. 테스트 해보기

In [194]:
# 가위 바위 보 3 폴더에 대해서 resize 진행
for dir in directories:   # rock, scissor, paper 3개 폴더에 대해서
    images = glob.glob(train_images_path + "/" + dir + "/*.jpg")
    for img in images:
        resize_image(img)   # resize
print("rock scissor paper resize finished")

#가위, 바위, 보 데이터를 읽을 수 있는 load_data() 함수로 데이터 불러오기
image_dir_path = os.getenv("HOME") + "/aiffel/rock_scissor_paper/test"
(x_test, y_test)=load_data(image_dir_path, 300) #x_test,y_test만들기
x_test_norm = x_test/255.0 

print("x_test shape: {}".format(x_test.shape))
print("y_test shape: {}".format(y_test.shape))

rock scissor paper resize finished
학습데이터(x_train)의 이미지 개수는 300 입니다.
x_test shape: (300, 122, 122, 3)
y_test shape: (300,)


In [195]:
test_loss, test_accuracy = model.evaluate(x_test, y_test, verbose = 2)

print("test_loss : {}".format(test_loss))
print("test_accuracy : {}".format(test_accuracy))

10/10 - 0s - loss: 0.0000e+00 - accuracy: 1.0000
test_loss : 0.0
test_accuracy : 1.0


# 회고

첫 익스인데도 너무 어렵게 느껴졌다. 어떻게 해야할지 모르겠어서 조원들이랑 얘기를 해봤다. 거기에서 여러가지 아이디어를 얻어 다양한 시도를 해보았다.

1. 데이터의 사이즈는 크게 키웠다.
2. 훈련 데이터셋의 양을 늘려봤다.
3. 하이퍼 파라미터의 값을 키웠다.
4. 원본 사진 자체의 환경을 잘 찍었다.(검은색 바탕에 손만 있게 해 봤다.)

위의 네가지를 모두 해본 결과 정확도가 모두 올랐다.

-과제를 진행하면서 아쉬웠던점-

1. 코드를 더 간결하게 짤 수는 없었나? 라는 생각이 든다. 더 간결하고 가독성도 좋게 할 수 있을 거 같은데 아쉽게 느껴진다.

2. 위의 4가지 말고도 정확도를 높이는 방법을 생각하지 않았다. 여러가지 방법이 더 있었을 텐데 과제를 다했다는 안도감에 더 시도해 보지않았다.

3. 원본 사진을 너무 깔끔하게 찍었다. 분명 실제 사용할 때는 좋은 환경에서 찍힌 데이터들이 거의 없을 텐데 좋은 환경을 배경으로 찍어버렸다. 과제라는 압박감이 있었나보다.

시간을 들여서 아쉬웠던 점에 대해서 더 생각하고 해결해 봐야되겠다.

2022-01-11